In [3]:
pip install khmerwordsegment

ERROR: Could not find a version that satisfies the requirement khmerwordsegment (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for khmerwordsegment
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from collections import Counter

# === Step 1: Provide your file path here ===
file_path = "/home/vitoupro/code/image_captioning/data/raw/rawimglabel.txt"  # 🔁 Replace with your actual path

# === Step 2: Load captions from the file ===
with open(file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

# Clean up: remove blank lines and trim whitespaces
captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Tokenize each caption into characters ===
# Word-level tokenization
tokenized_words = [caption.split() for caption in captions]

# === Step 4: Build vocabulary with special tokens ===
special_tokens = ['<PAD>', '<START>', '<END>', '<UNK>']
# Build vocab
all_words = [word for caption in tokenized_words for word in caption]
word_counts = Counter(all_words)
vocab_words = special_tokens + sorted(word_counts.keys())


word2idx_words = {word: idx for idx, word in enumerate(vocab_words)}
idx2word_words = {idx: word for word, idx in word2idx_words.items()}

# === Step 5: Convert captions to ID sequences ===
# Convert to ID sequences
def caption_to_ids_words(word_list):
    return [word2idx_words['<START>']] + [word2idx_words.get(word, word2idx_words['<UNK>']) for word in word_list] + [word2idx_words['<END>']]

sequences_words = [caption_to_ids_words(words) for words in tokenized_words]

# === Step 6: Pad sequences ===
max_len = max(len(seq) for seq in sequences_words)

def pad_sequence(seq, max_len):
    return seq + [word2idx_words['<PAD>']] * (max_len - len(seq))


# Padding
max_len_words = max(len(seq) for seq in sequences_words)
padded_sequences_words = [pad_sequence(seq, max_len_words) for seq in sequences_words]
padded_sequences_words = np.array(padded_sequences_words)

# === Step 7: Preview results ===
print(f"✅ Total captions: {len(captions)}")
print(f"✅ Vocabulary size: {len(all_words)}")
print(f"✅ Max sequence length: {max_len}")
print("\n📌 Sample Caption:", captions[0])
print("🧩 Tokenized:", tokenized_words[0])
print("🔢 Sequence:", sequences_words[0])
print("📏 Padded:", padded_sequences_words[0])


✅ Total captions: 425
✅ Vocabulary size: 425
✅ Max sequence length: 3

📌 Sample Caption: កូនពែងពណ៌ខ្មៅបួននិងកំសៀវពណ៌ខ្មៅមួយ
🧩 Tokenized: ['កូនពែងពណ៌ខ្មៅបួននិងកំសៀវពណ៌ខ្មៅមួយ']
🔢 Sequence: [1, 30, 2]
📏 Padded: [ 1 30  2]


In [ ]:
import json

# Save word2idx_wordleveltest5k and idx2word_wordleveltest5k to a file
with open("word2idx_words.json", "w", encoding='utf-8') as f:
    json.dump(word2idx_words, f, ensure_ascii=False, indent=4)

with open("idx2word_words.json", "w", encoding='utf-8') as f:
    json.dump(idx2word_words, f, ensure_ascii=False, indent=4)

print("✅ Vocabulary saved to word2idx_words.json and idx2word_words.json")


✅ Vocabulary saved to word2idx_words.json and idx2word_words.json


In [ ]:
import os
import json
import numpy as np
from collections import Counter

# Khmer Word Segmenter
class KhmerWordSegmenter:
    def __init__(self, dict_path):
        self.word_dict = set()
        self.max_word_len = 0
        self.load_dictionary(dict_path)

    def load_dictionary(self, dict_path):
        with open(dict_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    self.word_dict.add(word)
                    if len(word) > self.max_word_len:
                        self.max_word_len = len(word)

    def segment(self, text):
        result = []
        index = 0
        text_length = len(text)
        
        while index < text_length:
            matched = False
            for size in range(self.max_word_len, 0, -1):
                if index + size > text_length:
                    continue
                candidate = text[index:index+size]
                if candidate in self.word_dict:
                    result.append(candidate)
                    index += size
                    matched = True
                    break
            if not matched:
                result.append(text[index])
                index += 1
        return result

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/data/raw/rawimglabel.txt'  # 🔁 Update if needed
dict_path = '/home/vitoupro/code/image_captioning/data/raw/dic.txt'  # 🔁 Your Khmer dictionary

# === Step 2: Load Captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Word Segmentation
segmenter = KhmerWordSegmenter(dict_path)

tokenized_captions = [segmenter.segment(caption) for caption in captions]

# === Step 4: Build Vocabulary
special_tokens = ['<PAD>', '<START>', '<END>', '<UNK>']
all_words = [word for caption in tokenized_captions for word in caption]
word_counts = Counter(all_words)

# Sort vocab by frequency (optional), or just alphabetically
vocab_words = special_tokens + sorted(word_counts.keys())

word2idx_wordleveltest5k = {word: idx for idx, word in enumerate(vocab_words)}
idx2word_wordleveltest5k = {idx: word for word, idx in word2idx_wordleveltest5k.items()}

# === Step 5: Convert Captions to ID Sequences
def caption_to_ids(word_list):
    return [word2idx_wordleveltest5k['<START>']] + [word2idx_wordleveltest5k.get(word, word2idx_wordleveltest5k['<UNK>']) for word in word_list] + [word2idx_wordleveltest5k['<END>']]

sequences = [caption_to_ids(words) for words in tokenized_captions]

# === Step 6: Padding
max_len = max(len(seq) for seq in sequences)

def pad_sequence(seq, max_len):
    return seq + [word2idx_wordleveltest5k['<PAD>']] * (max_len - len(seq))

padded_sequences = [pad_sequence(seq, max_len) for seq in sequences]
padded_sequences = np.array(padded_sequences)

# === Step 7: Save Vocabulary
save_dir = '/home/vitoupro/code/image_captioning/data/processed/'
os.makedirs(save_dir, exist_ok=True)

with open(os.path.join(save_dir, 'word2idx_wordleveltest5k.json'), 'w', encoding='utf-8') as f:
    json.dump(word2idx_wordleveltest5k, f, ensure_ascii=False, indent=4)

with open(os.path.join(save_dir, 'idx2word_wordleveltest5k.json'), 'w', encoding='utf-8') as f:
    json.dump(idx2word_wordleveltest5k, f, ensure_ascii=False, indent=4)

print(f"✅ Finished!")
print(f"✅ Total captions: {len(captions)}")
print(f"✅ Vocabulary size (including special tokens): {len(word2idx_wordleveltest5k)}")
print(f"✅ Max sequence length: {max_len}")
print(f"✅ Saved word2idx_wordleveltest5k.json and idx2word_wordleveltest5k.json into {save_dir}")

# === Step 8: Preview
print("\n📌 Example caption:", captions[0])
print("🧩 Tokenized:", tokenized_captions[0])
print("🔢 Sequence:", sequences[0])
print("📏 Padded:", padded_sequences[0])


✅ Finished!
✅ Total captions: 425
✅ Vocabulary size (including special tokens): 481
✅ Max sequence length: 28
✅ Saved word2idx_wordlevel.json and idx2word_wordlevel.json into /home/vitoupro/code/image_captioning/data/processed/

📌 Example caption: កូនពែងពណ៌ខ្មៅបួននិងកំសៀវពណ៌ខ្មៅមួយ
🧩 Tokenized: ['កូន', 'ពែង', 'ពណ៌ខ្មៅ', 'បួន', 'និង', 'កំសៀវ', 'ពណ៌ខ្មៅ', 'មួយ']
🔢 Sequence: [1, 26, 333, 310, 274, 242, 38, 310, 353, 2]
📏 Padded: [  1  26 333 310 274 242  38 310 353   2   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [ ]:
import json

# Save word2idx_wordleveltest5k and idx2word_wordleveltest5k to a file
with open("word2idx_wordleveltest5k.json", "w", encoding='utf-8') as f:
    json.dump(word2idx_wordleveltest5k, f, ensure_ascii=False, indent=4)

with open("idx2word_level.json", "w", encoding='utf-8') as f:
    json.dump(idx2word_wordleveltest5k, f, ensure_ascii=False, indent=4)

print("✅ Vocabulary saved to word2idx_wordleveltest5k.json and idx2word_wordleveltest5k.json")

✅ Vocabulary saved to word2idx_wordlevel.json and idx2word_wordlevel.json


In [10]:
import os
import json
from collections import Counter

# Khmer Word Segmenter
class KhmerWordSegmenter:
    def __init__(self, dict_path):
        self.word_dict = set()
        self.max_word_len = 0
        self.load_dictionary(dict_path)

    def load_dictionary(self, dict_path):
        with open(dict_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    self.word_dict.add(word)
                    if len(word) > self.max_word_len:
                        self.max_word_len = len(word)

    def segment(self, text):
        result = []
        index = 0
        text_length = len(text)
        
        while index < text_length:
            matched = False
            for size in range(self.max_word_len, 0, -1):
                if index + size > text_length:
                    continue
                candidate = text[index:index+size]
                if candidate in self.word_dict:
                    result.append(candidate)
                    index += size
                    matched = True
                    break
            if not matched:
                result.append(text[index])
                index += 1
        return result

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/data/raw/imglabel.txt'  # your current label
dict_path = '/home/vitoupro/code/image_captioning/data/raw/dic.txt'  # your Khmer dictionary
output_label_path = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel.txt'  # ⬅️ NEW segmented label

# === Step 2: Load raw captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Initialize segmenter
segmenter = KhmerWordSegmenter(dict_path)

# === Step 4: Process and segment
segmented_lines = []
for line in captions:
    parts = line.strip().split(' ', 1)  # split only once (image and caption)
    if len(parts) != 2:
        continue
    image_name, caption = parts
    segmented_words = segmenter.segment(caption)
    segmented_caption = ' '.join(segmented_words)  # join back with space
    segmented_lines.append(f"{image_name} {segmented_caption}")

# === Step 5: Save new segmented label file
os.makedirs(os.path.dirname(output_label_path), exist_ok=True)

with open(output_label_path, 'w', encoding='utf-8') as f:
    for line in segmented_lines:
        f.write(line + '\n')

print(f"✅ Word-segmented captions saved to {output_label_path}")

✅ Word-segmented captions saved to /home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel.txt


In [36]:
import os
import json
from collections import Counter

# Khmer Word Segmenter
class KhmerWordSegmenter:
    def __init__(self, dict_path):
        self.word_dict = set()
        self.max_word_len = 0
        self.load_dictionary(dict_path)

    def load_dictionary(self, dict_path):
        with open(dict_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    self.word_dict.add(word)
                    if len(word) > self.max_word_len:
                        self.max_word_len = len(word)

    def segment(self, text):
        result = []
        index = 0
        text_length = len(text)
        
        while index < text_length:
            matched = False
            for size in range(self.max_word_len, 0, -1):
                if index + size > text_length:
                    continue
                candidate = text[index:index+size]
                if candidate in self.word_dict:
                    result.append(candidate)
                    index += size
                    matched = True
                    break
            if not matched:
                result.append(text[index])
                index += 1
        return result

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/data/raw/imglabel.txt'  # your current label
dict_path = '/home/vitoupro/code/image_captioning/data/raw/gptdic.txt'  # your Khmer dictionary
output_label_path = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel_gpt.txt'  # ⬅️ NEW segmented label

# === Step 2: Load raw captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Initialize segmenter
segmenter = KhmerWordSegmenter(dict_path)

# === Step 4: Process and segment
segmented_lines = []
for line in captions:
    parts = line.strip().split(' ', 1)  # split only once (image and caption)
    if len(parts) != 2:
        continue
    image_name, caption = parts
    segmented_words = segmenter.segment(caption)
    segmented_caption = ' '.join(segmented_words)  # join back with space
    segmented_lines.append(f"{image_name} {segmented_caption}")

# === Step 5: Save new segmented label file
os.makedirs(os.path.dirname(output_label_path), exist_ok=True)

with open(output_label_path, 'w', encoding='utf-8') as f:
    for line in segmented_lines:
        f.write(line + '\n')

print(f"✅ Word-segmented captions saved to {output_label_path}")

✅ Word-segmented captions saved to /home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel_gpt.txt


In [ ]:
#dic kloun eng
# Build rough dictionary from captions themselves
captions_file_path = '/home/vitoupro/code/image_captioning/data/raw/imglabel.txt'
output_dict_path = '/home/vitoupro/code/image_captioning/data/raw/dic_auto_generated.txt'

captions = []
with open(captions_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            _, caption = parts
            captions.append(caption)

# Naive split: treat every continuous 2, 3, 4, 5, 6 character groups as possible words
generated_words = set()

for caption in captions:
    for size in range(1, 8):  # try 2 to 6 character words
        for i in range(len(caption) - size + 1):
            substring = caption[i:i+size]
            generated_words.add(substring)

# Save the rough dictionary
with open(output_dict_path, 'w', encoding='utf-8') as f:
    for word in sorted(generated_words):
        f.write(word + '\n')

print(f"✅ Rough dictionary generated with {len(generated_words)} words!")


✅ Rough dictionary generated with 22409 words!


In [9]:
# Check captions properly
captions_file_path = '/home/vitoupro/code/image_captioning/data/raw/rawimglabel.txt'

with open(captions_file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

print(f"✅ Total lines: {len(lines)}\n")

for i, line in enumerate(lines[:10]):  # show first 10 lines
    print(f"[Line {i+1}] {line.strip()}")


✅ Total lines: 425

[Line 1] កូនពែងពណ៌ខ្មៅបួននិងកំសៀវពណ៌ខ្មៅមួយ
[Line 2] ឆ្កែអង្គុយលើសាឡុង
[Line 3] មានមួកនៅលើវ៉ាលីពណ៌ខ្មៅ
[Line 4] ថូពណ៌សមួយនៅជិតកៅអីនិងមានថូពណ៌សមួយទៀតនៅលើកៅអី
[Line 5] កំសៀវនៅលើតុចង្ក្រានបាយ
[Line 6] កន្ត្រកពណ៌ស
[Line 7] មានកាំបិតស្រួចនិងផ្លែស្តបឺរីនៅក្នុងកន្ត្រក
[Line 8] ឆ្កែពាក់មួកអង្គុយជិតវ៉ាលីពណ៌លឿង
[Line 9] កៅអីពីរនៅជិតគ្នានិងមានខ្នើយនៅលើកៅអីទាំងពីរ
[Line 10] ប្រអប់បាយពណ៌ផ្កាឈូក


In [33]:
import json
import os

# Load segmented captions
segmented_label_path = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel.txt'

captions = []
with open(segmented_label_path, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            _, caption = parts
            captions.append(caption)

# Tokenize into words
tokenized_captions = [caption.split() for caption in captions]

# Build vocabulary
special_tokens = ['<PAD>', '<START>', '<END>', '<UNK>']
all_words = [word for caption in tokenized_captions for word in caption]

# 🛠 Correct filtering: keep only words with length > 1
filtered_words = [word for word in set(all_words) if len(word) > 1]

# 🛠 Build vocab from filtered words
vocab_words = special_tokens + sorted(filtered_words)

# Create word2idx and idx2word
word2idx = {word: idx for idx, word in enumerate(vocab_words)}
idx2word = {idx: word for word, idx in word2idx.items()}

# Save
save_dir = '/home/vitoupro/code/image_captioning/data/processed/'
os.makedirs(save_dir, exist_ok=True)

with open(os.path.join(save_dir, 'word2idx_level.json'), 'w', encoding='utf-8') as f:
    json.dump(word2idx, f, ensure_ascii=False, indent=4)

with open(os.path.join(save_dir, 'idx2word_level.json'), 'w', encoding='utf-8') as f:
    json.dump(idx2word, f, ensure_ascii=False, indent=4)

print(f"✅ Saved cleaned word2idx_level.json and idx2word_level.json!")
print(f"✅ Vocabulary size: {len(word2idx)}")


✅ Saved cleaned word2idx_level.json and idx2word_level.json!
✅ Vocabulary size: 433


In [35]:
import re

# 🛠 Step 1: Define function to fix Khmer word segmentation
def fix_khmer_segmentation(text):
    words = text.strip().split()

    # Khmer unicode range
    khmer_range = re.compile(r'[\u1780-\u17FF]+')

    fixed_words = []
    for word in words:
        if not khmer_range.match(word):
            # Not a full Khmer word => likely broken vowel/diacritic => merge with previous
            if fixed_words:
                fixed_words[-1] += word
            else:
                fixed_words.append(word)
        else:
            fixed_words.append(word)
    return ' '.join(fixed_words)

# 🛠 Step 2: Apply it to fix the whole caption file
input_file = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel.txt'
output_file = '/home/vitoupro/code/image_captioning/data/processed/fixed_word_segmented_imglabel.txt'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = [line.strip() for line in f if line.strip()]

fixed_lines = []
for line in lines:
    parts = line.split(' ', 1)
    if len(parts) == 2:
        img_name, caption = parts
        fixed_caption = fix_khmer_segmentation(caption)
        fixed_lines.append(f"{img_name} {fixed_caption}")

# 🛠 Step 3: Save the fixed captions
with open(output_file, 'w', encoding='utf-8') as f:
    for line in fixed_lines:
        f.write(line + '\n')

print(f"✅ Finished fixing segmentation! Saved new file to: {output_file}")


✅ Finished fixing segmentation! Saved new file to: /home/vitoupro/code/image_captioning/data/processed/fixed_word_segmented_imglabel.txt


In [ ]:
import os
import json
import numpy as np
from collections import Counter

# Khmer Word Segmenter
class KhmerWordSegmenter:
    def __init__(self, dict_path):
        self.word_dict = set()
        self.max_word_len = 0
        self.load_dictionary(dict_path)

    def load_dictionary(self, dict_path):
        with open(dict_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    self.word_dict.add(word)
                    if len(word) > self.max_word_len:
                        self.max_word_len = len(word)

    def segment(self, text):
        result = []
        index = 0
        text_length = len(text)
        
        while index < text_length:
            matched = False
            for size in range(self.max_word_len, 0, -1):
                if index + size > text_length:
                    continue
                candidate = text[index:index+size]
                if candidate in self.word_dict:
                    result.append(candidate)
                    index += size
                    matched = True
                    break
            if not matched:
                result.append(text[index])
                index += 1
        return result

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/notebook/translated_dataset.txt'  # 🔁 Update if needed
dict_path = '/home/vitoupro/code/image_captioning/data/raw/dic.txt'  # 🔁 Your Khmer dictionary

# === Step 2: Load Captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Word Segmentation
segmenter = KhmerWordSegmenter(dict_path)

tokenized_captions = [segmenter.segment(caption) for caption in captions]

# === Step 4: Build Vocabulary
special_tokens = ['<PAD>', '<START>', '<END>', '<UNK>']
all_words = [word for caption in tokenized_captions for word in caption]
word_counts = Counter(all_words)

# Sort vocab by frequency (optional), or just alphabetically
vocab_words = special_tokens + sorted(word_counts.keys())

word2idx_wordleveltest5k = {word: idx for idx, word in enumerate(vocab_words)}
idx2word_wordleveltest5k = {idx: word for word, idx in word2idx_wordleveltest5k.items()}

# === Step 5: Convert Captions to ID Sequences
def caption_to_ids(word_list):
    return [word2idx_wordleveltest5k['<START>']] + [word2idx_wordleveltest5k.get(word, word2idx_wordleveltest5k['<UNK>']) for word in word_list] + [word2idx_wordleveltest5k['<END>']]

sequences = [caption_to_ids(words) for words in tokenized_captions]

# === Step 6: Padding
max_len = max(len(seq) for seq in sequences)

def pad_sequence(seq, max_len):
    return seq + [word2idx_wordleveltest5k['<PAD>']] * (max_len - len(seq))

padded_sequences = [pad_sequence(seq, max_len) for seq in sequences]
padded_sequences = np.array(padded_sequences)

# === Step 7: Save Vocabulary
save_dir = '/home/vitoupro/code/image_captioning/data/processed/'
os.makedirs(save_dir, exist_ok=True)

with open(os.path.join(save_dir, 'word2idx_wordleveltest5k.json'), 'w', encoding='utf-8') as f:
    json.dump(word2idx_wordleveltest5k, f, ensure_ascii=False, indent=4)

with open(os.path.join(save_dir, 'idx2word_wordleveltest5k.json'), 'w', encoding='utf-8') as f:
    json.dump(idx2word_wordleveltest5k, f, ensure_ascii=False, indent=4)

print(f"✅ Finished!")
print(f"✅ Total captions: {len(captions)}")
print(f"✅ Vocabulary size (including special tokens): {len(word2idx_wordleveltest5k)}")
print(f"✅ Max sequence length: {max_len}")
print(f"✅ Saved word2idx_wordleveltest5k.json and idx2word_wordleveltest5k.json into {save_dir}")

# === Step 8: Preview
print("\n📌 Example caption:", captions[0])
print("🧩 Tokenized:", tokenized_captions[0])
print("🔢 Sequence:", sequences[0])
print("📏 Padded:", padded_sequences[0])


✅ Finished!
✅ Total captions: 998
✅ Vocabulary size (including special tokens): 2262
✅ Max sequence length: 185
✅ Saved word2idx_wordleveltest.json and idx2word_wordleveltest.json into /home/vitoupro/code/image_captioning/data/processed/

📌 Example caption: image_1.jpg ក្មេងប្រុសពីរនាក់ដែលមានសក់រញ៉េរញ៉ៃកំពុងមើលទៅដៃរបស់ពួកគេ ខណៈពួកគេកំពុងអង្គុយលេងនៅក្នុងវាល។
🧩 Tokenized: ['i', 'm', 'a', 'g', 'e', '_', '1', '.', 'j', 'p', 'g', ' ', 'ក្មេងប្រុស', 'ពីរ', 'នាក់', 'ដែលមាន', 'សក', '់', 'រញ៉េរញ៉ៃ', 'កំពុង', 'មើលទៅ', 'ដៃ', 'របស់', 'ពួកគេ', ' ', 'ខណៈ', 'ពួកគេ', 'កំពុង', 'អង្គុយលេង', 'នៅក្នុង', 'វ', 'ា', 'ល', '។']
🔢 Sequence: [1, 47, 51, 39, 45, 43, 38, 8, 6, 48, 54, 45, 4, 264, 1394, 1038, 737, 1841, 2251, 1637, 226, 1570, 738, 1659, 1411, 4, 303, 1411, 226, 2097, 1071, 1803, 2230, 1739, 2257, 2]
📏 Padded: [   1   47   51   39   45   43   38    8    6   48   54   45    4  264
 1394 1038  737 1841 2251 1637  226 1570  738 1659 1411    4  303 1411
  226 2097 1071 1803 2230 1739 2257    2    0    0

In [7]:
import os
import json
from collections import Counter

# Khmer Word Segmenter
class KhmerWordSegmenter:
    def __init__(self, dict_path):
        self.word_dict = set()
        self.max_word_len = 0
        self.load_dictionary(dict_path)

    def load_dictionary(self, dict_path):
        with open(dict_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    self.word_dict.add(word)
                    if len(word) > self.max_word_len:
                        self.max_word_len = len(word)

    def segment(self, text):
        result = []
        index = 0
        text_length = len(text)
        
        while index < text_length:
            matched = False
            for size in range(self.max_word_len, 0, -1):
                if index + size > text_length:
                    continue
                candidate = text[index:index+size]
                if candidate in self.word_dict:
                    result.append(candidate)
                    index += size
                    matched = True
                    break
            if not matched:
                result.append(text[index])
                index += 1
        return result

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/notebook/translated_dataset.txt'  # your current label
dict_path = '/home/vitoupro/code/image_captioning/data/raw/dic.txt'  # your Khmer dictionary
output_label_path = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabeltest.txt'  # ⬅️ NEW segmented label

# === Step 2: Load raw captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Initialize segmenter
segmenter = KhmerWordSegmenter(dict_path)

# === Step 4: Process and segment
segmented_lines = []
for line in captions:
    parts = line.strip().split(' ', 1)  # split only once (image and caption)
    if len(parts) != 2:
        continue
    image_name, caption = parts
    segmented_words = segmenter.segment(caption)
    segmented_caption = ' '.join(segmented_words)  # join back with space
    segmented_lines.append(f"{image_name} {segmented_caption}")

# === Step 5: Save new segmented label file
os.makedirs(os.path.dirname(output_label_path), exist_ok=True)

with open(output_label_path, 'w', encoding='utf-8') as f:
    for line in segmented_lines:
        f.write(line + '\n')

print(f"✅ Word-segmented captions saved to {output_label_path}")

✅ Word-segmented captions saved to /home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabeltest.txt


In [3]:
#for 5k
import os
import json
import numpy as np
from collections import Counter

# Khmer Word Segmenter
class KhmerWordSegmenter:
    def __init__(self, dict_path):
        self.word_dict = set()
        self.max_word_len = 0
        self.load_dictionary(dict_path)

    def load_dictionary(self, dict_path):
        with open(dict_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    self.word_dict.add(word)
                    if len(word) > self.max_word_len:
                        self.max_word_len = len(word)

    def segment(self, text):
        result = []
        index = 0
        text_length = len(text)
        
        while index < text_length:
            matched = False
            for size in range(self.max_word_len, 0, -1):
                if index + size > text_length:
                    continue
                candidate = text[index:index+size]
                if candidate in self.word_dict:
                    result.append(candidate)
                    index += size
                    matched = True
                    break
            if not matched:
                result.append(text[index])
                index += 1
        return result

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/notebook/translated_dataset5k.txt'  # 🔁 Your label file
dict_path = '/home/vitoupro/code/image_captioning/data/raw/dic.txt'  # 🔁 Your Khmer dictionary
save_dir = '/home/vitoupro/code/image_captioning/data/processed/'  # 🔁 Where to save output
os.makedirs(save_dir, exist_ok=True)

# === Step 2: Load Captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_lines = f.readlines()

# === Step 3: Split out image names and captions
image_names = []
captions = []

for line in raw_lines:
    line = line.strip()
    if not line:
        continue
    parts = line.split(' ', 1)  # split into image name and caption
    if len(parts) == 2:
        image_name, caption = parts
        image_names.append(image_name)
        captions.append(caption)
    else:
        print(f"⚠️ Skipping malformed line: {line}")

# === Step 4: Word Segmentation (on captions only)
segmenter = KhmerWordSegmenter(dict_path)

tokenized_captions = [segmenter.segment(caption) for caption in captions]

# === Step 5: Build Vocabulary
special_tokens = ['<PAD>', '<START>', '<END>', '<UNK>']
all_words = [word for caption in tokenized_captions for word in caption]
word_counts = Counter(all_words)

# Sort vocab by frequency (optional) or alphabetically
vocab_words = special_tokens + sorted(word_counts.keys())

word2idx_wordleveltest5k = {word: idx for idx, word in enumerate(vocab_words)}
idx2word_wordleveltest5k = {idx: word for word, idx in word2idx_wordleveltest5k.items()}

# === Step 6: Convert Captions to ID Sequences
def caption_to_ids(word_list):
    return [word2idx_wordleveltest5k['<START>']] + [word2idx_wordleveltest5k.get(word, word2idx_wordleveltest5k['<UNK>']) for word in word_list] + [word2idx_wordleveltest5k['<END>']]

sequences = [caption_to_ids(words) for words in tokenized_captions]

# === Step 7: Padding
max_len = max(len(seq) for seq in sequences)

def pad_sequence(seq, max_len):
    return seq + [word2idx_wordleveltest5k['<PAD>']] * (max_len - len(seq))

padded_sequences = [pad_sequence(seq, max_len) for seq in sequences]
padded_sequences = np.array(padded_sequences)

# === Step 8: Save Vocabulary
with open(os.path.join(save_dir, 'word2idx_wordleveltest5k.json'), 'w', encoding='utf-8') as f:
    json.dump(word2idx_wordleveltest5k, f, ensure_ascii=False, indent=4)

with open(os.path.join(save_dir, 'idx2word_wordleveltest5k.json'), 'w', encoding='utf-8') as f:
    json.dump(idx2word_wordleveltest5k, f, ensure_ascii=False, indent=4)

print(f"✅ Finished!")
print(f"✅ Total captions: {len(captions)}")
print(f"✅ Vocabulary size (including special tokens): {len(word2idx_wordleveltest5k)}")
print(f"✅ Max sequence length: {max_len}")
print(f"✅ Saved word2idx_wordleveltest5k.json and idx2word_wordleveltest5k.json into {save_dir}")

# === Step 9: Preview
print("\n📌 Example image:", image_names[0])
print("📌 Example caption:", captions[0])
print("🧩 Tokenized:", tokenized_captions[0])
print("🔢 Sequence:", sequences[0])
print("📏 Padded:", padded_sequences[0])


✅ Finished!
✅ Total captions: 4989
✅ Vocabulary size (including special tokens): 4227
✅ Max sequence length: 171
✅ Saved word2idx_wordleveltest5k.json and idx2word_wordleveltest5k.json into /home/vitoupro/code/image_captioning/data/processed/

📌 Example image: image_1.jpg
📌 Example caption: បុរសវ័យក្មេងពីរនាក់ដែលមានសក់ខ្មាស់អៀនមើលទៅដៃរបស់ពួកគេនៅពេលដើរលេងនៅទីធ្លា
🧩 Tokenized: ['បុរស', 'វ័យក្មេង', 'ពីរ', 'នាក់', 'ដែលមាន', 'សក', '់', 'ខ្មាស', '់', 'អៀន', 'មើលទៅ', 'ដៃ', 'របស់', 'ពួកគេ', 'នៅពេល', 'ដើរលេង', 'នៅ', 'ទីធ្លា']
🔢 Sequence: [1, 2132, 3400, 2535, 1891, 1340, 3402, 4219, 628, 4219, 4053, 2847, 1341, 3025, 2562, 1971, 1320, 1947, 1672, 2]
📏 Padded: [   1 2132 3400 2535 1891 1340 3402 4219  628 4219 4053 2847 1341 3025
 2562 1971 1320 1947 1672    2    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0  

In [6]:
#for 5k
import os
import json
from collections import Counter

# Khmer Word Segmenter
class KhmerWordSegmenter:
    def __init__(self, dict_path):
        self.word_dict = set()
        self.max_word_len = 0
        self.load_dictionary(dict_path)

    def load_dictionary(self, dict_path):
        with open(dict_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    self.word_dict.add(word)
                    if len(word) > self.max_word_len:
                        self.max_word_len = len(word)

    def segment(self, text):
        result = []
        index = 0
        text_length = len(text)
        
        while index < text_length:
            matched = False
            for size in range(self.max_word_len, 0, -1):
                if index + size > text_length:
                    continue
                candidate = text[index:index+size]
                if candidate in self.word_dict:
                    result.append(candidate)
                    index += size
                    matched = True
                    break
            if not matched:
                result.append(text[index])
                index += 1
        return result

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/notebook/translated_dataset5k.txt'  # your current label
dict_path = '/home/vitoupro/code/image_captioning/data/raw/dic.txt'  # your Khmer dictionary
output_label_path = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabeltest5k.txt'  # ⬅️ NEW segmented label

# === Step 2: Load raw captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Initialize segmenter
segmenter = KhmerWordSegmenter(dict_path)

# === Step 4: Process and segment
segmented_lines = []
for line in captions:
    parts = line.strip().split(' ', 1)  # split only once (image_name and caption)
    if len(parts) != 2:
        print(f"⚠️ Warning: Skipping malformed line: {line}")
        continue
    
    image_name, caption = parts
    segmented_words = segmenter.segment(caption)
    segmented_caption = ' '.join(segmented_words)  # join back with space
    segmented_line = f"{image_name} {segmented_caption}"
    segmented_lines.append(segmented_line)

# === Step 5: Save new segmented label file
os.makedirs(os.path.dirname(output_label_path), exist_ok=True)

with open(output_label_path, 'w', encoding='utf-8') as f:
    for line in segmented_lines:
        f.write(line + '\n')

print(f"✅ Word-segmented captions saved to {output_label_path}")
print(f"✅ Total lines processed: {len(segmented_lines)}")


✅ Word-segmented captions saved to /home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabeltest5k.txt
✅ Total lines processed: 4989


In [1]:
pip install khmercut

  Using cached khmercut-0.1.0.tar.gz (5.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for khmercut: filename=khmercut-0.1.0-py3-none-any.whl size=5872135 sha256=fab439dc37439034490153ebd2859a5d8c5122573cf226b167384902d473ace0
  Stored in directory: /home/vitoupro/.cache/pip/wheels/11/56/60/586253cc7205c610ea0644cff5e1e0c1ce57798756c2f1ab9e
Successfully built khmercut

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from khmercut import tokenize
import os

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/notebook/translated_dataset5k.txt'  # raw label
output_label_path = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabeltest5k_khmercut.txt'  # ⬅️ NEW segmented label

# === Step 2: Load raw captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Segment each caption using khmercut.tokenize
segmented_lines = []
for line in captions:
    parts = line.strip().split(' ', 1)
    if len(parts) != 2:
        print(f"⚠️ Skipping malformed line: {line}")
        continue

    image_name, caption = parts
    segmented_words = tokenize(caption)  # ✅ Use khmercut's tokenizer here
    segmented_caption = ' '.join(segmented_words)
    segmented_lines.append(f"{image_name} {segmented_caption}")

# === Step 4: Save segmented result
os.makedirs(os.path.dirname(output_label_path), exist_ok=True)

with open(output_label_path, 'w', encoding='utf-8') as f:
    for line in segmented_lines:
        f.write(line + '\n')

print(f"✅ Word-segmented captions saved to {output_label_path}")
print(f"✅ Total lines processed: {len(segmented_lines)}")


✅ Word-segmented captions saved to /home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabeltest5k_khmercut.txt
✅ Total lines processed: 4989


In [4]:
import os
import json

# === Step 1: Load segmented caption file
segmented_file_path = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabeltest5k_khmercut.txt'

captions = []
with open(segmented_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            _, caption = parts
            captions.append(caption)

# === Step 2: Tokenize into word list
tokenized_captions = [caption.split() for caption in captions]
all_words = [word for caption in tokenized_captions for word in caption]

# === Step 3: Create vocab with special tokens
special_tokens = ['<PAD>', '<START>', '<END>', '<UNK>']
vocab_words = special_tokens + sorted(set(all_words))

word2idx = {word: idx for idx, word in enumerate(vocab_words)}
idx2word = {idx: word for word, idx in word2idx.items()}

# === Step 4: Save vocab files
save_dir = '/home/vitoupro/code/image_captioning/data/processed/'
os.makedirs(save_dir, exist_ok=True)

with open(os.path.join(save_dir, 'word2idx_level_khmercut.json'), 'w', encoding='utf-8') as f:
    json.dump(word2idx, f, ensure_ascii=False, indent=4)

with open(os.path.join(save_dir, 'idx2word_level_khmercut.json'), 'w', encoding='utf-8') as f:
    json.dump(idx2word, f, ensure_ascii=False, indent=4)

print("✅ Vocabulary saved to:")
print("→ word2idx_level_khmercut.json")
print("→ idx2word_level_khmercut.json")
print(f"🟩 Total unique words (excluding special tokens): {len(vocab_words) - len(special_tokens)}")


✅ Vocabulary saved to:
→ word2idx_level_khmercut.json
→ idx2word_level_khmercut.json
🟩 Total unique words (excluding special tokens): 4066


In [5]:
from khmercut import tokenize
import os

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/data/raw/imglabel.txt'  # raw label
output_label_path = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel_khmercut.txt'  # ⬅️ NEW segmented label

# === Step 2: Load raw captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Segment each caption using khmercut.tokenize
segmented_lines = []
for line in captions:
    parts = line.strip().split(' ', 1)
    if len(parts) != 2:
        print(f"⚠️ Skipping malformed line: {line}")
        continue

    image_name, caption = parts
    segmented_words = tokenize(caption)  # ✅ Use khmercut's tokenizer here
    segmented_caption = ' '.join(segmented_words)
    segmented_lines.append(f"{image_name} {segmented_caption}")

# === Step 4: Save segmented result
os.makedirs(os.path.dirname(output_label_path), exist_ok=True)

with open(output_label_path, 'w', encoding='utf-8') as f:
    for line in segmented_lines:
        f.write(line + '\n')

print(f"✅ Word-segmented captions saved to {output_label_path}")
print(f"✅ Total lines processed: {len(segmented_lines)}")


✅ Word-segmented captions saved to /home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel_khmercut.txt
✅ Total lines processed: 425


In [6]:
import os
import json

# === Step 1: Load segmented caption file
segmented_file_path = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel_khmercut.txt'

captions = []
with open(segmented_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            _, caption = parts
            captions.append(caption)

# === Step 2: Tokenize into word list
tokenized_captions = [caption.split() for caption in captions]
all_words = [word for caption in tokenized_captions for word in caption]

# === Step 3: Create vocab with special tokens
special_tokens = ['<PAD>', '<START>', '<END>', '<UNK>']
vocab_words = special_tokens + sorted(set(all_words))

word2idx = {word: idx for idx, word in enumerate(vocab_words)}
idx2word = {idx: word for word, idx in word2idx.items()}

# === Step 4: Save vocab files
save_dir = '/home/vitoupro/code/image_captioning/data/processed/'
os.makedirs(save_dir, exist_ok=True)

with open(os.path.join(save_dir, 'word2idx_level_khmercut_420.json'), 'w', encoding='utf-8') as f:
    json.dump(word2idx, f, ensure_ascii=False, indent=4)

with open(os.path.join(save_dir, 'idx2word_level_khmercut_420.json'), 'w', encoding='utf-8') as f:
    json.dump(idx2word, f, ensure_ascii=False, indent=4)

print("✅ Vocabulary saved to:")
print("→ word2idx_level_khmercut.json")
print("→ idx2word_level_khmercut.json")
print(f"🟩 Total unique words (excluding special tokens): {len(vocab_words) - len(special_tokens)}")


✅ Vocabulary saved to:
→ word2idx_level_khmercut.json
→ idx2word_level_khmercut.json
🟩 Total unique words (excluding special tokens): 371
